
# Extractive Question Answering

This notebook demonstrates how Pinecone helps you build an extractive question-answering application. To build an extractive question-answering system, we need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A reader model to extract answers

We will use the SQuAD dataset, which consists of **questions** and **context** paragraphs containing question **answers**. We generate embeddings for the context passages using the retriever, index them in the vector database, and query with semantic search to retrieve the top k most relevant contexts containing potential answers to our question. We then use the reader model to extract the answers from the returned contexts.

In [ ]:
pip install pinecone-client


Let's get started by installing the packages needed for notebook to run:

# Install Dependencies

In [ ]:
!pip install -qU datasets pinecone-client sentence-transformers torch

# Load Dataset

Now let's load the SQUAD dataset from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

In [ ]:
from datasets import load_dataset

# load the squad dataset into a pandas dataframe
df = load_dataset("squad", split="train").to_pandas()

In [ ]:
# select only title and context column
df = df[["title", "context"]]
# drop rows containing duplicate context passages
df = df.drop_duplicates(subset="context")
df

,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
5,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
10,University_of_Notre_Dame,The university is the major seat of the Congre...
15,University_of_Notre_Dame,The College of Engineering was established in ...
20,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...
...,...,...
87574,Kathmandu,"Institute of Medicine, the central college of ..."
87579,Kathmandu,Football and Cricket are the most popular spor...
87584,Kathmandu,The total length of roads in Nepal is recorded...
87589,Kathmandu,The main international airport serving Kathman...


In [ ]:
len(df)

18891

# Initialize Pinecone Index

The Pinecone index stores vector representations of our context passages which we can retrieve using another vector (query vector). We first need to initialize our connection to Pinecone to create our vector index. For this, we need a free [API key]("https://app.pinecone.io/"), and then we initialize the connection like so:

In [ ]:
from getpass import getpass

PINECONE_API_KEY = getpass()

··········


In [ ]:
import pinecone

# connect to pinecone environment
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='us-east-1-aws'  # find next to API key in console
)

Now we create a new index called "question-answering" — we can name the index anything we want. We specify the metric type as "cosine" and dimension as 384 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 384-dimension vectors.

In [ ]:
index_name = "extractive-question-answering"

# check if the extractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=384,
        metric="cosine"
    )

# connect to extractive-question-answering index we created
index = pinecone.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will generate embeddings in a way that the questions and context passages containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and context embeddings to find the context passages that contain potential answers to our question.

We will use a SentenceTransformer model named ``multi-qa-MiniLM-L6-cos-v1`` designed for semantic search and trained on 215M (question, answer) pairs from diverse sources as our retriever.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, context passage, etc.

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch['context'].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient='records')
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/296 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 18891}},
 'total_vector_count': 18891}

In [ ]:
to_upsert[-2]


('18889',
 [0.09583466500043869,
  -0.0027207592502236366,
  0.00136305030900985,
  -0.004785728175193071,
  -0.019396422430872917,
  0.0131625821813941,
  0.04574994370341301,
  0.013086422346532345,
  -0.0049654701724648476,
  -0.013829726725816727,
  -0.0017555875238031149,
  -0.05788721889257431,
  -0.07169235497713089,
  0.03153254836797714,
  0.0312524251639843,
  0.05953897908329964,
  0.09483731538057327,
  0.015777934342622757,
  -0.05010640248656273,
  -0.0032226925250142813,
  0.10360261797904968,
  0.005556721705943346,
  -0.04196544364094734,
  0.01238032802939415,
  0.038373399525880814,
  0.0077400910668075085,
  0.03352518379688263,
  -0.022155022248625755,
  0.038833171129226685,
  -0.039342451840639114,
  0.011614843271672726,
  -0.08203130960464478,
  -0.08203326165676117,
  0.05163583159446716,
  -0.029236402362585068,
  0.00643193582072854,
  -0.11927835643291473,
  0.07092610001564026,
  0.054749276489019394,
  -0.010198121890425682,
  0.041892070323228836,
  0.00

In [ ]:
len(to_upsert[-2][1])

384

# Initialize Reader

We use the `deepset/electra-base-squad2` model from the HuggingFace model hub as our reader model. We load this model into a "question-answering" pipeline from HuggingFace transformers and feed it our questions and context passages individually. The model gives a prediction for each context we pass through the pipeline.

In [ ]:
from transformers import pipeline

model_name = 'deepset/electra-base-squad2'
# load the reader model into a question-answering pipeline
reader = pipeline(tokenizer=model_name, model=model_name, task='question-answering', device='cuda:0')
reader

Now all the components we need are ready. Let's write some helper functions to execute our queries. The `get_context` function retrieves the context embeddings containing answers to our question from the Pinecone index, and the `extract_answer` function extracts the answers from these context passages.

In [ ]:
# gets context passages from the pinecone index
def get_context(question, top_k):
    # generate embeddings for the question
    xq = retriever.encode([question]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    # extract the context passage from pinecone search result
    c = [x["metadata"]['context'] for x in xc["matches"]]
    return c

In [ ]:
from pprint import pprint

# extracts answer from the context passage
def extract_answer(question, context):
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c)
        # add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)
    # sort the result based on the score from reader model
    sorted_result = pprint(sorted(results, key=lambda x: x['score'], reverse=True))
    return sorted_result

In [ ]:
question = "How much oil is Egypt producing in a day?"
context = get_context(question, top_k = 1)
context

['Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of natural gas (in 2013), which makes Egypt as the largest oil producer not member of the Organization of the Petroleum Exporting Countries (OPEC) and the second-largest dry natural gas producer in Africa. In 2013, Egypt was the largest consumer of oil and natural gas in Africa, as more than 20% of total oil consumption and more than 40% of total dry natural gas consumption in Africa. Also, Egypt possesses the largest oil refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is currently planning to build its first nuclear power plant in El Dabaa city, northern Egypt.']

As we can see, the retiever is working fine and gets us the context passage that contains the answer to our question. Now let's use the reader to extract the exact answer from the context passage.

In [ ]:
extract_answer(question, context)

[{'answer': '691,000 bbl/d',
  'context': 'Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of '
             'natural gas (in 2013), which makes Egypt as the largest oil '
             'producer not member of the Organization of the Petroleum '
             'Exporting Countries (OPEC) and the second-largest dry natural '
             'gas producer in Africa. In 2013, Egypt was the largest consumer '
             'of oil and natural gas in Africa, as more than 20% of total oil '
             'consumption and more than 40% of total dry natural gas '
             'consumption in Africa. Also, Egypt possesses the largest oil '
             'refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is '
             'currently planning to build its first nuclear power plant in El '
             'Dabaa city, northern Egypt.',
  'end': 33,
  'score': 0.9999852180480957,
  'start': 20}]


The reader model predicted with 99% accuracy the correct answer *691,000 bbl/d* as seen from the context passage. Let's run few more queries.

In [ ]:
question = "What are the first names of the men that invented youtube?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Hurley and Chen',
  'context': 'According to a story that has often been repeated in the media, '
             'Hurley and Chen developed the idea for YouTube during the early '
             'months of 2005, after they had experienced difficulty sharing '
             "videos that had been shot at a dinner party at Chen's apartment "
             'in San Francisco. Karim did not attend the party and denied that '
             'it had occurred, but Chen commented that the idea that YouTube '
             'was founded after a dinner party "was probably very strengthened '
             'by marketing ideas around creating a story that was very '
             'digestible".',
  'end': 79,
  'score': 0.9999276399612427,
  'start': 64}]


In [ ]:
question = "What is Albert Eistein famous for?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'his theories of special relativity and general relativity',
  'context': 'Albert Einstein is known for his theories of special relativity '
             'and general relativity. He also made important contributions to '
             'statistical mechanics, especially his mathematical treatment of '
             'Brownian motion, his resolution of the paradox of specific '
             'heats, and his connection of fluctuations and dissipation. '
             'Despite his reservations about its interpretation, Einstein also '
             'made contributions to quantum mechanics and, indirectly, quantum '
             'field theory, primarily through his theoretical studies of the '
             'photon.',
  'end': 86,
  'score': 0.95003741979599,
  'start': 29}]


Let's run another question. This time for top 3 context passages from the retriever.

In [ ]:
question = "Who was the first person to step foot on the moon?"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'Armstrong',
  'context': 'The trip to the Moon took just over three days. After achieving '
             'orbit, Armstrong and Aldrin transferred into the Lunar Module, '
             'named Eagle, and after a landing gear inspection by Collins '
             'remaining in the Command/Service Module Columbia, began their '
             'descent. After overcoming several computer overload alarms '
             'caused by an antenna switch left in the wrong position, and a '
             'slight downrange error, Armstrong took over manual flight '
             'control at about 180 meters (590 ft), and guided the Lunar '
             'Module to a safe landing spot at 20:18:04 UTC, July 20, 1969 '
             '(3:17:04 pm CDT). The first humans on the Moon would wait '
             'another six hours before they ventured out of their craft. At '
             '02:56 UTC, July 21 (9:56 pm CDT July 20), Armstrong became the '
             'first human to set foot on the Moon.',

The result looks pretty good.

# Team Orange Contribution

- As the following queries suggest, it would be desired to have more level of reasoning over the contexts that we have queried. Following implementation

- to have a better search that can compare different documents that have different entities. Haven't figured out yet

In [ ]:
question = "How tall is the Temple of Sagrada Familia"
context = get_context(question, top_k=3)
context

['The synagogue in Eshtemoa (As-Samu) was built around the 4th century. The mosaic floor is decorated with only floral and geometric patterns. The synagogue in Khirbet Susiya (excavated in 1971–72, founded in the end of the 4th century) has three mosaic panels, the eastern one depicting a Torah shrine, two menorahs, a lulav and an etrog with columns, deer and rams. The central panel is geometric while the western one is seriously damaged but it has been suggested that it depicted Daniel in the lion’s den. The Roman synagogue in Ein Gedi was remodeled in the Byzantine era and a more elaborate mosaic floor was laid down above the older white panels. The usual geometric design was enriched with birds in the center. It includes the names of the signs of the zodiac and important figures from the Jewish past but not their images suggesting that it served a rather conservative community.',
 "A series of low-lying annexes (largely hidden) flank both ends. Also in the square are the glass-faced

In [ ]:
question = "What is the Population in Spain"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'with a population ranging from 1.7 to 2.5 million',
  'context': 'Valencia (/vəˈlɛnsiə/; Spanish: [baˈlenθja]), or València '
             '(Valencian: [vaˈlensia]), is the capital of the autonomous '
             'community of Valencia and the third largest city in Spain after '
             'Madrid and Barcelona, with around 800,000 inhabitants in the '
             'administrative centre. Its urban area extends beyond the '
             'administrative city limits with a population of around 1.5 '
             "million people. Valencia is Spain's third largest metropolitan "
             'area, with a population ranging from 1.7 to 2.5 million. The '
             'city has global city status. The Port of Valencia is the 5th '
             'busiest container port in Europe and the busiest container port '
             'on the Mediterranean Sea.',
  'end': 476,
  'score': 0.19772616028785706,
  'start': 427},
 {'answer': 'Valencia has a population of 809,267',
  'context'

### Chatbot PoC GPT-3

In [ ]:
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 5.4 MB/s eta 0:00:00


In [ ]:
### Chatbot PoC GPT-3
import openai
import pandas as pd
import numpy as np
from getpass import getpass

openai.api_key = getpass()

··········


In [ ]:
question = "How tall is the Temple of Sagrada Familia"
context = get_context(question, top_k=3)
context = "\n\n".join(context)
pprint(context)

('The synagogue in Eshtemoa (As-Samu) was built around the 4th century. The '
 'mosaic floor is decorated with only floral and geometric patterns. The '
 'synagogue in Khirbet Susiya (excavated in 1971–72, founded in the end of the '
 '4th century) has three mosaic panels, the eastern one depicting a Torah '
 'shrine, two menorahs, a lulav and an etrog with columns, deer and rams. The '
 'central panel is geometric while the western one is seriously damaged but it '
 'has been suggested that it depicted Daniel in the lion’s den. The Roman '
 'synagogue in Ein Gedi was remodeled in the Byzantine era and a more '
 'elaborate mosaic floor was laid down above the older white panels. The usual '
 'geometric design was enriched with birds in the center. It includes the '
 'names of the signs of the zodiac and important figures from the Jewish past '
 'but not their images suggesting that it served a rather conservative '
 'community.\n'
 '\n'
 'A series of low-lying annexes (largely hidden) 

In [ ]:
import openai
import os


# Define function to generate text using the OpenAI API
def generate_text(prompt, model="text-davinci-003", max_tokens=1024, temperature=0.5):
    """
    Generates text using the OpenAI API.

    Arguments:
    prompt -- The prompt to use for generating text.
    model -- The name of the model to use (default "davinci").
    max_tokens -- The maximum number of tokens to generate (default 1024).
    temperature -- The sampling temperature to use (default 0.5).

    Returns:*
    The generated text.
    """

    # Call the OpenAI API to generate text
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature
    )

    # Get the generated text from the API response
    generated_text = response.choices[0].text

    return generated_text

In [ ]:
prompt = '''You are an AI assistant that answers questions to a user.
You are given the following extracted parts of a long document as "Context" and a question.
Use the Context to answer the Question. If the relevant information for the question is not in the answer please say that "The information in the queried context is not relevant for the User Question Provided".

Context: "{}"

Question: "{}"

Answer :'''.format(context, question+"?")


In [ ]:
print(prompt)

You are an AI assistant that answers questions to a user.
You are given the following extracted parts of a long document as "Context" and a question.
Use the Context to answer the Question. If the relevant information for the question is not in the answer please say that "The information in the queried context is not relevant for the User Question Provided".

Context: "The synagogue in Eshtemoa (As-Samu) was built around the 4th century. The mosaic floor is decorated with only floral and geometric patterns. The synagogue in Khirbet Susiya (excavated in 1971–72, founded in the end of the 4th century) has three mosaic panels, the eastern one depicting a Torah shrine, two menorahs, a lulav and an etrog with columns, deer and rams. The central panel is geometric while the western one is seriously damaged but it has been suggested that it depicted Daniel in the lion’s den. The Roman synagogue in Ein Gedi was remodeled in the Byzantine era and a more elaborate mosaic floor was laid down abov

In [ ]:
generated_text = generate_text(prompt, temperature=0.1)


In [ ]:
print(generated_text)

 The information in the queried context is not relevant for the User Question Provided.


## More Spice!, thoughful answers for convoluted questions

In [ ]:
question = "Who won the Korean War and Why?"
context = get_context(question, top_k=5)
context = "\n\n".join(context)
pprint(context)

('The Korean War (in South Korean Hangul: 한국전쟁, Hanja: 韓國戰爭, Hanguk Jeonjaeng, '
 '"Korean War"; in North Korean Chosungul: 조국해방전쟁, Joguk Haebang Jeonjaeng, '
 '"Fatherland Liberation War"; 25 June 1950 – 27 July 1953)[a] was started '
 'when North Korea invaded South Korea. The United Nations, with United States '
 'as the principal force, came to aid of South Korea. China, along with '
 'assistance from Soviet Union, came to aid of North Korea. The war arose from '
 'the division of Korea at the end of World War II and from the global '
 'tensions of the Cold War that developed immediately afterwards.\n'
 '\n'
 'The Korean War was a conflict between the United States and its United '
 'Nations allies and the communist powers under influence of the Soviet Union '
 "(also a UN member nation) and the People's Republic of China (which later "
 'also gained UN membership). The principal combatants were North and South '
 'Korea. Principal allies of South Korea included the United States, 

In [ ]:
prompt = '''You are an AI assistant that answers questions to a user.
You are given the following extracted parts of a long document as "Context" and a question.
Use the Context to answer the Question. If the relevant information for the question is not in the answer please say that "The information in the queried context is not relevant for the User Question Provided".

Context: "{}"

Question: "{}"

Answer :'''.format(context, question+"?")
generated_text = generate_text(prompt, temperature=0.5)


In [ ]:
pprint(generated_text)

(' The Korean War ended in a stalemate, with neither side gaining a clear '
 'victory. The war was ended with the signing of the Korean Armistice '
 'Agreement in 1953. Both sides agreed to a ceasefire, with the front line '
 'remaining close to where it was at the start of the war. The agreement also '
 'established the Korean Demilitarized Zone (DMZ) between the two countries.')


## ChatGPT API

In [ ]:
question = "List the best triomphs of Napoleon in bulletpoints"
context = get_context(question, top_k=5)
context = "\n\n".join(context)
pprint(context)

('During the Napoleonic Wars he was taken seriously by the British press as a '
 'dangerous tyrant, poised to invade. He was often referred to by the British '
 'as Boney. A nursery rhyme warned children that Bonaparte ravenously ate '
 'naughty people; the "bogeyman". The British Tory press sometimes depicted '
 'Napoleon as much smaller than average height, and this image persists. '
 'Confusion about his height also results from the difference between the '
 'French pouce and British inch—2.71 cm and 2.54 cm, respectively. The myth of '
 'the "Napoleon Complex” — named after him to describe men who have an '
 'inferiority complex — stems primarily from the fact that he was listed, '
 'incorrectly, as 5 feet 2 inches (in French units) at the time of his death. '
 'In fact, he was 1.68 metres (5 ft 6 in) tall, an average height for a man in '
 'that period.[note 11]\n'
 '\n'
 "Historians agree that Napoleon's remarkable personality was one key to his "
 'influence. They emphasize the 

In [ ]:
prompt = '''You are an AI assistant that answers questions to a user.
You are given the following extracted parts of a long document as "Context" and a question.
Use the Context to answer the Question. If the relevant information for the question is not in the answer please say that "The information in the queried context is not relevant for the User Question Provided".

Context: "{}"'''.format(context)


In [ ]:
pprint(prompt)

('You are an AI assistant that answers questions to a user.\n'
 'You are given the following extracted parts of a long document as "Context" '
 'and a question.\n'
 'Use the Context to answer the Question. If the relevant information for the '
 'question is not in the answer please say that "The information in the '
 'queried context is not relevant for the User Question Provided".\n'
 '\n'
 'Context: "During the Napoleonic Wars he was taken seriously by the British '
 'press as a dangerous tyrant, poised to invade. He was often referred to by '
 'the British as Boney. A nursery rhyme warned children that Bonaparte '
 'ravenously ate naughty people; the "bogeyman". The British Tory press '
 'sometimes depicted Napoleon as much smaller than average height, and this '
 'image persists. Confusion about his height also results from the difference '
 'between the French pouce and British inch—2.71 cm and 2.54 cm, respectively. '
 'The myth of the "Napoleon Complex” — named after him to desc

In [ ]:
from IPython.display import Markdown

def print_markdown(text):
    """
    Prints Markdown-formatted text to the console.
    """
    display(Markdown(text))

In [ ]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work

ChatGPT_Response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": question}
    ]
)

In [ ]:
print_markdown(ChatGPT_Response['choices'][0]['message']['content'])

- During the Italian campaign, Napoleon's army captured 150,000 prisoners, 540 cannons, and 170 standards. The French army fought 67 actions and won 18 pitched battles through superior artillery technology and Bonaparte's tactics.
- French forces managed to capture Vienna in November, providing the French a huge bounty as they captured 100,000 muskets, 500 cannons, and the intact bridges across the Danube.
- At the Battle of Austerlitz, in Moravia on 2 December, he deployed the French army below the Pratzen Heights and deliberately weakened his right flank, enticing the Allies to launch a major assault there in the hopes of rolling up the whole French line. With the Allied center demolished, the French swept through both enemy flanks and sent the Allies fleeing chaotically, capturing thousands of prisoners in the process. The battle is often seen as a tactical masterpiece.

### Further Work

- Need To work more at prompt engineer Level
- Need for the Vector Search to be effective
- There is a tradeoff between the payload of 0-shot/few-shot Learning and Fine-Tuning, that insure a more consistent style
- Need to figure out how to query dissimilar entitiest (Need to implement entity extraction along with the vector search?)
- Use of Langchain?

